In [ ]:
from doctest import DocFileCase
import yfinance as yf

# Choose an interval: '1d' (daily), '1wk' (weekly), '1mo' (monthly), etc.
interval = '3mo'  # Change this to '1wk' or '1mo' as needed
symbol = "^GSPC"
# Download historical data for S&P 500 (^GSPC) with the chosen interval
df = yf.download(symbol, period="1y", interval=interval)

print(f"Downloaded S&P 500 data with interval: {interval}")

df


/tmp/ipykernel_20033/3913302791.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol, period="1y", interval=interval)
[*********************100%***********************]  1 of 1 completed

Downloaded S&P 500 data with interval: 3mo


Price,Close,High,Low,Open,Volume
Ticker,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC
Date,,,,,
2024-12-01,5954.500000,6147.430176,5773.310059,5903.259766,180956380000
2025-03-01,5911.689941,5986.089844,4835.040039,5968.330078,335669910000
2025-06-01,6460.259766,6508.229980,5861.430176,5896.680176,319813220000
2025-09-01,6849.089844,6920.339844,6360.580078,6401.509766,344616440000
2025-12-01,6870.029785,6895.779785,6799.939941,6812.299805,20189896000


In [2]:
from lib.line import Calculator


calculator = Calculator()






In [5]:
import pandas as pd
import numpy as np

# Calculate dots first (needed for 6-x methods)
dots_valid = calculator.calculate_dots(df, symbol)
print(f"Calculated {len(dots_valid)} valid dots")

# Start with a copy of the original dataframe
result_df = df.copy()

# Add dots column - merge with dots_valid
if isinstance(result_df.columns, pd.MultiIndex):
    # For MultiIndex, we need to handle it differently
    dots_series = pd.Series(index=df.index, dtype=float)
    dots_series.loc[dots_valid.index] = dots_valid['dots'].values
    result_df[('Dots', symbol)] = dots_series
else:
    dots_series = pd.Series(index=df.index, dtype=float)
    dots_series.loc[dots_valid.index] = dots_valid['dots'].values
    result_df['Dots'] = dots_series

# Define all resistance and support function names
line_functions = [
    ('5_2_resistance', calculator.get_5_2_resistance, False),
    ('5_2_support', calculator.get_5_2_support, False),
    ('5_1_resistance', calculator.get_5_1_resistance, False),
    ('5_1_support', calculator.get_5_1_support, False),
    ('5_3_resistance', calculator.get_5_3_resistance, False),
    ('5_3_support', calculator.get_5_3_support, False),
    ('5_9_resistance', calculator.get_5_9_resistance, False),
    ('5_9_support', calculator.get_5_9_support, False),
    ('6_1_resistance', calculator.get_6_1_resistance, True),
    ('6_1_support', calculator.get_6_1_support, True),
    ('6_5_resistance', calculator.get_6_5_resistance, True),
    ('6_5_support', calculator.get_6_5_support, True),
    ('6_7_resistance', calculator.get_6_7_resistance, True),
    ('6_7_support', calculator.get_6_7_support, True),
]

# Initialize columns for all resistance/support types
for col_name, _, _ in line_functions:
    if isinstance(result_df.columns, pd.MultiIndex):
        result_df[(col_name, symbol)] = np.nan
    else:
        result_df[col_name] = np.nan

# Initialize binary target column (1 if price goes up next, 0 if down)
if isinstance(result_df.columns, pd.MultiIndex):
    result_df[('price_up', symbol)] = np.nan
else:
    result_df['price_up'] = np.nan

# Get dots as Series for 6-x functions
if isinstance(dots_valid, pd.DataFrame):
    dots_series_for_calc = dots_valid['dots']
else:
    dots_series_for_calc = dots_valid

# Iterate through all valid indices (need at least 3 bars, so start from index 2)
num_rows = len(df)
print(f"\nCalculating resistances and supports for {num_rows - 2} indices...")

for idx in range(2, num_rows):
    # Use negative index to work from the end
    index = -(num_rows - idx)
    
    # Calculate all lines for this index
    for col_name, func, needs_dots in line_functions:
        try:
            if needs_dots:
                point_1, point_2 = func(df, dots_series_for_calc, symbol, index)
            else:
                point_1, point_2 = func(df, symbol, index)
            
            # Store point2 y-value if line exists
            if point_1 is not None and point_2 is not None:
                y_value = point_2[1]  # y-value of point2
                # Get the date index for this bar
                if isinstance(result_df.columns, pd.MultiIndex):
                    result_df.loc[df.index[idx], (col_name, symbol)] = y_value
                else:
                    result_df.loc[df.index[idx], col_name] = y_value
        except Exception as e:
            # Skip if calculation fails (bounds, etc.)
            pass

# Calculate binary target: 1 if price goes up next, 0 if down
# Get close prices
if isinstance(result_df.columns, pd.MultiIndex):
    close_col = result_df[('Close', symbol)]
else:
    close_col = result_df['Close']

# For each row, compare current close with next close
for idx in range(len(result_df) - 1):
    current_close = close_col.iloc[idx]
    next_close = close_col.iloc[idx + 1]
    
    # 1 if price goes up, 0 if down
    price_up_value = 1 if next_close > current_close else 0
    
    if isinstance(result_df.columns, pd.MultiIndex):
        result_df.iloc[idx, result_df.columns.get_loc(('price_up', symbol))] = price_up_value
    else:
        result_df.iloc[idx, result_df.columns.get_loc('price_up')] = price_up_value

# Last row will remain NaN (no next row to compare)

print("Calculation complete!")
print(f"\nResult DataFrame shape: {result_df.shape}")
print(f"\nColumns: {list(result_df.columns)}")
print(f"\nPrice up distribution:")
if isinstance(result_df.columns, pd.MultiIndex):
    print(result_df[('price_up', symbol)].value_counts())
else:
    print(result_df['price_up'].value_counts())
print(f"\nFirst few rows:")
print(result_df.head(10))
print(f"\nLast few rows:")
print(result_df.tail(10))
result_df

Calculated 248 valid dots

Calculating resistances and supports for 248 indices...


/tmp/ipykernel_20033/1037122791.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[np.float64(6078.085503472223) np.float64(6065.601128472223)
 np.float64(6061.902180989583) np.float64(6061.034450954861)
 np.float64(6064.949978298611) np.float64(6062.892252604167)
 np.float64(6058.592230902777) np.float64(6019.167805989583)
 np.float64(5958.085557725694) np.float64(5913.833333333333)
 np.float64(5918.752170138889) np.float64(5962.408854166667)
 np.float64(6001.2421875) np.float64(6007.361111111111)
 np.float64(5969.062228732639) np.float64(5923.008897569444)
 np.float64(5892.25439453125) np.float64(5899.31884765625)
 np.float64(5930.007703993056) np.float64(5948.587782118056)
 np.float64(5941.971082899306) np.float64(5894.053331163194)
 np.float64(5854.914442274306) np.float64(5832.617784288194)
 np.float64(5864.902235243056) np.float64(5907.636664496527)
 np.float64(5959.837782118056) np.float64(5992

Calculation complete!

Result DataFrame shape: (250, 21)

Columns: [('Close', '^GSPC'), ('High', '^GSPC'), ('Low', '^GSPC'), ('Open', '^GSPC'), ('Volume', '^GSPC'), ('Dots', '^GSPC'), ('5_2_resistance', '^GSPC'), ('5_2_support', '^GSPC'), ('5_1_resistance', '^GSPC'), ('5_1_support', '^GSPC'), ('5_3_resistance', '^GSPC'), ('5_3_support', '^GSPC'), ('5_9_resistance', '^GSPC'), ('5_9_support', '^GSPC'), ('6_1_resistance', '^GSPC'), ('6_1_support', '^GSPC'), ('6_5_resistance', '^GSPC'), ('6_5_support', '^GSPC'), ('6_7_resistance', '^GSPC'), ('6_7_support', '^GSPC'), ('price_up', '^GSPC')]

Price up distribution:
(price_up, ^GSPC)
1.0    142
0.0    107
Name: count, dtype: int64

First few rows:
Price             Close         High          Low         Open      Volume  \
Ticker            ^GSPC        ^GSPC        ^GSPC        ^GSPC       ^GSPC   
Date                                                                         
2024-12-05  6075.109863  6094.549805  6072.899902  6089.029785  421

Price,Close,High,Low,Open,Volume,Dots,5_2_resistance,5_2_support,5_1_resistance,5_1_support,...,5_3_support,5_9_resistance,5_9_support,6_1_resistance,6_1_support,6_5_resistance,6_5_support,6_7_resistance,6_7_support,price_up
Ticker,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC,...,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
2024-12-05,6075.109863,6094.549805,6072.899902,6089.029785,4212020000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2024-12-06,6090.270020,6099.970215,6079.979980,6081.379883,3924830000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2024-12-09,6052.850098,6088.509766,6048.629883,6083.009766,4556460000,6078.085503,6105.390625,NaN,NaN,NaN,...,6087.060059,6127.040527,6065.410156,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2024-12-10,6034.910156,6065.399902,6029.890137,6057.589844,4048410000,6065.601128,NaN,6017.279785,NaN,NaN,...,NaN,6097.039551,5997.289551,6107.541124,NaN,6067.661241,NaN,NaN,NaN,1.0
2024-12-11,6084.189941,6092.589844,6060.149902,6060.149902,4269950000,6061.902181,NaN,6011.150391,NaN,NaN,...,NaN,6082.169922,5971.270508,6101.312120,NaN,6065.802355,NaN,6065.802355,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-11-28,6849.089844,6850.859863,6819.750000,6822.520020,2558540000,6794.431044,6886.479980,NaN,6907.760254,NaN,...,NaN,7002.899902,6791.340332,NaN,6653.393392,NaN,6700.963216,NaN,6700.963216,0.0
2025-12-01,6812.629883,6843.649902,6799.939941,6812.299805,4549370000,6822.648817,6870.279785,NaN,6855.629883,NaN,...,NaN,6917.849609,6808.060059,NaN,6738.002224,NaN,6769.112088,NaN,6769.112088,1.0
